# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of each sentence from `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function

    source_sentences = source_text.split("\n")
    source_sentences = [sentence.split(" ") for sentence in source_sentences]
    # remove '' to avoid errors
    source_sentences = [[word for word in sentence if word != ''] for sentence in source_sentences]
    source_id_text = [ [ source_vocab_to_int[word] for word in sentence] for sentence in source_sentences]
    
    target_sentences = target_text.split("\n")
    target_sentences = [sentence.split(" ") for sentence in target_sentences]
    # remove '' to avoid errors
    target_sentences = [[word for word in sentence if word != ''] for sentence in target_sentences]
    target_id_text = [ [ target_vocab_to_int[word] for word in sentence] for sentence in target_sentences]
    target_id_text = [ sentence + [target_vocab_to_int['<EOS>']] for sentence in target_id_text]    
    #target_id_text = [ [target_vocab_to_int['<GO>']] + sentence + [target_vocab_to_int['<EOS>']] for sentence in target_id_text]    
    #target_id_text = [ [target_vocab_to_int['<GO>']] + sentence for sentence in target_id_text]    

    return (source_id_text, target_id_text)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
#tests.test_text_to_ids(text_to_ids)

"\nDON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE\n"

### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) in [LooseVersion('1.0.0'), LooseVersion('1.0.1')], 'This project requires TensorFlow version 1.0  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1


/Users/simonflachs/anaconda/envs/dlnd-tf-lab/lib/python3.5/site-packages/ipykernel/__main__.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoding_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.

Return the placeholders in the following the tuple (Input, Targets, Learing Rate, Keep Probability)

In [7]:
def model_inputs(batch_size):
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate, keep probability)
    """
    input = tf.placeholder(tf.int32, shape=(batch_size,None), name='input')
    targets = tf.placeholder(tf.int32, shape=(batch_size,None), name='targets')
    
    learning_rate = tf.placeholder(tf.float32, shape=(), name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, shape=(), name='keep_prob')
    # TODO: Implement Function
    return input, targets, learning_rate, keep_prob

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
#tests.test_model_inputs(model_inputs)

"\nDON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE\n"

### Process Decoding Input
Implement `process_decoding_input` using TensorFlow to remove the last word id from each batch in `target_data` and concat the GO ID to the beginning of each batch.

In [8]:
def process_decoding_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for decoding
    :param target_data: Target Placeholder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """

    # remove the last word id from each batch
    target_data = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])

    # concat go id to beginning of each batch
    GO_id = target_vocab_to_int['<GO>']    
    GO_id_col = tf.fill([batch_size, 1], GO_id)
    target_data = tf.concat([GO_id_col, target_data], axis=1)
        
    return target_data

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
#tests.test_process_decoding_input(process_decoding_input)

"\nDON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE\n"

### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer using [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn).

In [9]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :return: RNN state
    """
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_size)
    
    lstm = tf.contrib.rnn.DropoutWrapper(lstm, keep_prob)
    
    cell = tf.contrib.rnn.MultiRNNCell([lstm] * num_layers)
    
    outputs, state = tf.nn.dynamic_rnn(cell, rnn_inputs, dtype=tf.float32)
    
    return state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create training logits using [`tf.contrib.seq2seq.simple_decoder_fn_train()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_train) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder).  Apply the `output_fn` to the [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder) outputs.

In [10]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param sequence_length: Sequence Length
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Train Logits
    """ 
    
    decoder_fn_train = tf.contrib.seq2seq.simple_decoder_fn_train(encoder_state)
     
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, keep_prob)
    
    outputs, final_state, final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(
        dec_cell, 
        decoder_fn_train,
        inputs = dec_embed_input,
        sequence_length=sequence_length,
        scope=decoding_scope
        )

    # TODO: Implement Function
    return output_fn(outputs), final_state#final_context_state#final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
#tests.test_decoding_layer_train(decoding_layer_train)

"\nDON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE\n"

### Decoding - Inference
Create inference logits using [`tf.contrib.seq2seq.simple_decoder_fn_inference()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_inference) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder). 

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param maximum_length: The maximum allowed time steps to decode
    :param vocab_size: Size of vocabulary
    :param decoding_scope: TensorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Inference Logits
    """
    """
    infer_fn = tf.contrib.seq2seq.simple_decoder_fn_inference(output_fn, encoder_state,
    dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size)
    drop = tf.contrib.rnn.DropoutWrapper(dec_cell, keep_prob)
    outputs, state, context = tf.contrib.seq2seq.dynamic_rnn_decoder(drop, infer_fn,
        sequence_length = maximum_length, scope = decoding_scope)
    inference_logits =  outputs
    return inference_logits
    """
    
    decoder_fn_inference = tf.contrib.seq2seq.simple_decoder_fn_inference(
        output_fn,
        encoder_state,
        dec_embeddings,
        start_of_sequence_id,
        end_of_sequence_id,
        maximum_length,
        vocab_size)

    
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, keep_prob)
    
    outputs, final_state, final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(
        dec_cell,
        decoder_fn_inference,
        sequence_length=maximum_length,
        scope=decoding_scope)
    
    # TODO: should i apply output_fn?
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
#tests.test_decoding_layer_infer(decoding_layer_infer)

"\nDON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE\n"

### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

- Create RNN cell for decoding using `rnn_size` and `num_layers`.
- Create the output fuction using [`lambda`](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) to transform it's input, logits, to class logits.
- Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob)` function to get the training logits.
- Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size, decoding_scope, output_fn, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def sample(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob, dec_embeddings, batch_size, chosen_ids = None):
    

    sequence_length= tf.ones(shape=(batch_size), dtype=tf.int32)
    max_output_length = 10
    
    # first input
    dec_input = tf.constant([[source_vocab_to_int['<GO>']]])
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    curr_input = tf.tile(dec_embed_input, [batch_size,1,1]) # replicate input to batch size

    # first state
    curr_state = encoder_state
    
    probs = []
    outputs = []

    for i in range(max_output_length):
        logits, curr_state = decoding_layer_train(curr_state, dec_cell, curr_input, sequence_length, decoding_scope, output_fn, keep_prob)
       
        dim = tf.reduce_prod(tf.shape(logits)[:2]) # remove max_len axis (2nd. axis), since it's 1
        logits_reshaped = tf.reshape(logits, [dim,-1])  
          
        if chosen_ids is None: # pick output from logits
            output_ids = tf.multinomial(logits_reshaped, 1)
            output_ids = tf.cast(output_ids, tf.int32)
        else: # pick pre-chosen ids, in order to get their probs
            output_ids = chosen_ids[:,i]
            output_ids = tf.expand_dims(output_ids, 1)
        
        # get prob of output
        softmax = tf.nn.log_softmax(logits_reshaped)  
        softmax_idxs = tf.range(tf.shape(softmax)[0])
        softmax_idxs = tf.expand_dims(softmax_idxs, 1) # match output_ids dimesions
        gather_idxs = tf.concat(( softmax_idxs, output_ids), axis=1)
        prob = tf.gather_nd(softmax, gather_idxs) 
     
        probs.append(prob)
        
        # update input
        curr_input = tf.nn.embedding_lookup(dec_embeddings, output_ids)

        outputs.append(tf.squeeze(output_ids))
        
    outputs = tf.stack(outputs, axis=1)
    probs = tf.stack(probs, axis=1)

    return outputs, probs

def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, target_vocab_to_int, keep_prob, batch_size, chosen_ids):
    """
    Create decoding layer
    :param dec_embed_input: Decoder embedded input
    :param dec_embeddings: Decoder embeddings
    :param encoder_state: The encoded state
    :param vocab_size: Size of vocabulary
    :param sequence_length: Sequence Length
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param keep_prob: Dropout keep probability
    :return: Tuple of (Training Logits, Inference Logits)
    """
    # TODO: Implement Function
    
    start_of_sequence_id =  target_vocab_to_int['<GO>']
    end_of_sequence_id = target_vocab_to_int['<EOS>']
    
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_size)
    lstm = tf.contrib.rnn.DropoutWrapper(lstm, keep_prob)
    dec_cell = tf.contrib.rnn.MultiRNNCell([lstm] * num_layers)
    
    output_fn = lambda x: tf.contrib.layers.fully_connected(x, vocab_size, None, scope=decoding_scope)
    
    with tf.variable_scope('decoding') as decoding_scope:
        training_logits, training_final_state = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob)

    with tf.variable_scope('decoding', reuse=True) as decoding_scope:
        inference_logits, inference_final_state = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, sequence_length, vocab_size, decoding_scope, output_fn, keep_prob)
    
    with tf.variable_scope('decoding', reuse=True) as decoding_scope:
        sample_out, probs = sample(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob, dec_embeddings, batch_size)
        
        _, predefined_probs = sample(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob, dec_embeddings, batch_size, chosen_ids)
    return training_logits, inference_logits, sample_out, probs, predefined_probs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
#tests.test_decoding_layer(decoding_layer)

"\nDON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE\n"

### Build the Neural Network
Apply the functions you implemented above to:

- Apply embedding to the input data for the encoder.
- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob)`.
- Process target data using your `process_decoding_input(target_data, target_vocab_to_int, batch_size)` function.
- Apply embedding to the target data for the decoder.
- Decode the encoded input using your `decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size, num_layers, target_vocab_to_int, keep_prob)`.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size, rnn_size, num_layers, target_vocab_to_int, chosen_ids):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param sequence_length: Sequence Length
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training Logits, Inference Logits)
    """
    
    # Apply embedding to the input data for the encoder.
    embedded_input = tf.contrib.layers.embed_sequence(
        input_data,
        vocab_size=source_vocab_size,
        embed_dim=enc_embedding_size
    )
    
    # Encode the input 
    encoded_state = encoding_layer(embedded_input, rnn_size, num_layers, keep_prob)
    #encoded_state_tensor = tf.identity(encoded_state, name='encoded_state')
    
    # Process target data 
    target_data = process_decoding_input(target_data, target_vocab_to_int, batch_size)
    
    # Apply embedding to the target data for the decoder.
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, dec_embedding_size]))  
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, target_data)
    
    # Decode the encoded input
    training_logits, inference_logits, sample, probs, predefined_probs = decoding_layer(
        dec_embed_input,
        dec_embeddings,
        encoded_state,
        target_vocab_size,
        sequence_length,
        rnn_size,
        num_layers,
        target_vocab_to_int,
        keep_prob, 
        batch_size,
        chosen_ids
    )
    
    return training_logits, inference_logits, sample, probs, predefined_probs

#tests.test_seq2seq_model(seq2seq_model)

## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability

In [14]:
# Number of Epochs
mle_epochs = 1
reinforce_epochs = 1

# Batch Size
mle_batch_size = 128
reinforce_batch_size = 64

# RNN Size
rnn_size = 256

# Number of Layers
num_layers = 2

# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128

# Learning Rate
mle_learning_rate = 0.001
reinforce_learning_rate = 0.000001

# Dropout Keep Probability
keep_probability = 0.8

# Save path
mle_save_path = 'checkpoints/mle/dev'
reinforce_save_path = 'checkpoints/reinforce/dev'

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_source_sentence_length = max([len(sentence) for sentence in source_int_text])

class RNN:
    
    def __init__(self, batch_size):

        #train_graph = tf.Graph()
        #with train_graph.as_default():
        tf.reset_default_graph()

        self.advantage = tf.placeholder(tf.float32, shape=(None), name='advantage')
        self.chosen_ids = tf.placeholder(tf.int32, shape=(batch_size,None), name='chosen_ids')

        self.input_data, self.targets, self.lr, self.keep_prob = model_inputs(batch_size)
        self.sequence_length = tf.placeholder_with_default(max_source_sentence_length, None, name='sequence_length')
        input_shape = tf.shape(self.input_data)

        self.train_logits, self.inference_logits, self.sample, self.probs, self.predefined_probs = seq2seq_model(
            tf.reverse(self.input_data, [-1]), self.targets, self.keep_prob, batch_size, self.sequence_length, len(source_vocab_to_int), len(target_vocab_to_int),
            encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, target_vocab_to_int, self.chosen_ids)

        tf.identity(self.inference_logits, 'logits')
        with tf.name_scope("optimization"):
   
            optimizer = tf.train.AdamOptimizer(self.lr)
            
            # Reinforce optimizer 1
            sample_probs = tf.reduce_sum(self.predefined_probs, axis=-1)    
            self.loss = -tf.reduce_mean(sample_probs*self.advantage)
            self.slow_train_op = optimizer.minimize(self.loss)
            
            # Reinforce optimizer 2
            # this should be faster if i can get it working
            self.sample_costs = - tf.reduce_sum(self.probs, axis=-1)         
            self.sample_gradients = optimizer.compute_gradients(self.sample_costs[0]) # do for all of batch
            self.new_train_op = optimizer.apply_gradients(self.sample_gradients, name='train_op')
            
            # MLE optimizer
            self.mle_cost = tf.contrib.seq2seq.sequence_loss(
                self.train_logits,
                self.targets,
                tf.ones([input_shape[0], self.sequence_length]))

            # Optimizer
            mle_optimizer = tf.train.AdamOptimizer(self.lr)

            # Gradient Clipping
            gradients = optimizer.compute_gradients(self.mle_cost)
            capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
            self.mle_train_op = optimizer.apply_gradients(capped_gradients)



## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
- Convert words not in the vocabulary, to the `<UNK>` word id.

In [16]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    sentence = sentence.lower()
    words = sentence.split(" ")
    return [vocab_to_int[word] if word in vocab_to_int.keys() else vocab_to_int['<UNK>'] for word in words ]

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forums to see if anyone is having the same problem.

In [18]:
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1]), (0,0)],
            'constant')

    return np.mean(np.equal(target, np.argmax(logits, 2)))

#temp
source_int_text = source_int_text#[:10000]
target_int_text = target_int_text#[:10000]

train_source = source_int_text[mle_batch_size:]
train_target = target_int_text[mle_batch_size:]

valid_source = helper.pad_sentence_batch(source_int_text[:mle_batch_size])
valid_target = helper.pad_sentence_batch(target_int_text[:mle_batch_size])



# MLE Training

In [19]:
import time

train_method = 'MLE'

model = RNN(mle_batch_size)

print("training...")

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch_i in range(mle_epochs):

        for batch_i, (source_batch, target_batch) in enumerate(helper.batch_data(train_source, train_target, mle_batch_size)):
            
            start_time = time.time()
                
            _, loss = sess.run(
                [model.mle_train_op, model.mle_cost],
                {model.input_data: source_batch,
                 model.targets: target_batch,
                 model.lr: mle_learning_rate,
                 model.sequence_length: target_batch.shape[1],
                 model.keep_prob: keep_probability})

            batch_train_logits = sess.run(
                model.inference_logits,
                {model.input_data: source_batch, model.keep_prob: 1.0})
            batch_valid_logits = sess.run(
                model.inference_logits,
                {model.input_data: valid_source, model.keep_prob: 1.0})

            train_acc = get_accuracy(target_batch, batch_train_logits)
            valid_acc = get_accuracy(np.array(valid_target), batch_valid_logits)
            end_time = time.time()
            
            output = train_method + ': Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.3f}, Validation Accuracy: {:>6.3f}, Loss: {:>6.3f}'.format(epoch_i, batch_i, len(source_int_text) // mle_batch_size, train_acc, valid_acc, loss)
            print(output)
            """
            if batch_i % 100: 
                translate_sentence = 'he saw a old yellow truck .'
                translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)
                translate_logits = sess.run(model.inference_logits, {model.input_data: [translate_sentence], model.keep_prob: 1.0})[0]
                print(' '.join(word for word in [target_int_to_vocab[i] for i in np.argmax(translate_logits, 1)] if word != '<PAD>'and word != '<EOS>'))
            """
        
    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, mle_save_path)
    print('Model Trained and Saved')
    
    


training...
MLE: Epoch   0 Batch    0/1077 - Train Accuracy:  0.294, Validation Accuracy:  0.305, Loss:  5.879
MLE: Epoch   0 Batch    1/1077 - Train Accuracy:  0.221, Validation Accuracy:  0.305, Loss:  5.531
MLE: Epoch   0 Batch    2/1077 - Train Accuracy:  0.206, Validation Accuracy:  0.305, Loss:  5.181
MLE: Epoch   0 Batch    3/1077 - Train Accuracy:  0.239, Validation Accuracy:  0.305, Loss:  4.667
MLE: Epoch   0 Batch    4/1077 - Train Accuracy:  0.225, Validation Accuracy:  0.305, Loss:  4.559
MLE: Epoch   0 Batch    5/1077 - Train Accuracy:  0.273, Validation Accuracy:  0.318, Loss:  4.272
MLE: Epoch   0 Batch    6/1077 - Train Accuracy:  0.276, Validation Accuracy:  0.332, Loss:  4.147
MLE: Epoch   0 Batch    7/1077 - Train Accuracy:  0.261, Validation Accuracy:  0.337, Loss:  4.035
MLE: Epoch   0 Batch    8/1077 - Train Accuracy:  0.269, Validation Accuracy:  0.337, Loss:  3.891
MLE: Epoch   0 Batch    9/1077 - Train Accuracy:  0.274, Validation Accuracy:  0.337, Loss:  3.78

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(mle_save_path)

# Reinforce Training

In [21]:
import nltk
from nltk.translate.bleu_score import SmoothingFunction

def get_reward(reference, hypothesis):
    return nltk.translate.bleu_score.sentence_bleu(reference,hypothesis, smoothing_function=SmoothingFunction().method4)

def get_advantages(samples, reference):
    
    if not isinstance(reference, str):
        reference_words = [target_int_to_vocab[j] for j in reference]
        reference = ' '.join(word for word in reference_words if word != '<PAD>'and word != '<EOS>')
    
    rewards = []
    for i in range(len(samples)):
        out = [target_int_to_vocab[j] for j in samples[i]]
        hypothesis = ' '.join(word for word in out)
        rewards.append(get_reward(reference, hypothesis))

    avg_reward = sum(rewards)/len(rewards)
    std_dev = np.array(rewards).std()

    advantage_input = np.array([(x-avg_reward)/std_dev for x in rewards])
    return advantage_input, avg_reward
    
def train_reinforce_on_sentence(model, max_length, n_samples, source, reference, learning_rate, keep_prob):

        samples_out, probs_out = sess.run(
            [model.sample, model.probs],
            {model.input_data: [source]*n_samples,
            model.keep_prob: keep_prob,
            model.lr: learning_rate})
        
        advantages_in, avg_reward = get_advantages(samples_out, reference)
        
        loss, _ = sess.run(
            [model.loss, model.slow_train_op] ,
            {model.input_data: [source]*n_samples,
             model.chosen_ids: samples_out,
             model.advantage: advantages_in,
            model.keep_prob: keep_prob,
            model.lr: learning_rate})

        
        """
        samples_out, probs_out, sample_gradients_out = sess.run(
            [model.sample, model.probs, model.sample_gradients],
            {model.input_data: [source]*n_samples,
            model.keep_prob: 1.,
            model.lr: 0.0001})
        
        
        print(sample_gradients_out)
        _ = sess.run(
            model.new_train_op,
            {model.sample_gradients: np.array(sample_gradients_out),
            model.keep_prob: 1.,
            model.lr: 0.0001})
        """
        return avg_reward, loss, samples_out, probs_out

def get_n_best_samples(n, samples, probs):
    sum_probs = np.sum(probs, axis=-1)
    top_idxs = sum_probs.argsort()[-n:][::-1]
    return samples[top_idxs], sum_probs[top_idxs]


In [22]:
import time

train_method = 'reinforce'
reinforce_epochs = 1
tf.reset_default_graph()
model = RNN(reinforce_batch_size)

print("training...")

checkpoint = tf.train.latest_checkpoint("checkpoints/mle")

saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, checkpoint)
        
    for epoch_i in range(reinforce_epochs):
        
        for batch_i, (source_batch, target_batch) in enumerate(helper.batch_data(train_source, train_target, reinforce_batch_size)):
            
            start_time = time.time()

            for source, target in zip(source_batch, target_batch):
                avg_reward, loss, samples, probability = train_reinforce_on_sentence(model, 10, reinforce_batch_size, source, target, reinforce_learning_rate, keep_probability)
            
            batch_train_logits = sess.run(
                model.inference_logits,
                {model.input_data: source_batch[:reinforce_batch_size], model.keep_prob: 1.0})
            batch_valid_logits = sess.run(
                model.inference_logits,
                {model.input_data: valid_source[:reinforce_batch_size], model.keep_prob: 1.0})

            train_acc = get_accuracy(target_batch[:reinforce_batch_size], batch_train_logits)
            valid_acc = get_accuracy(np.array(valid_target[:reinforce_batch_size]), batch_valid_logits)
            end_time = time.time()
            
            output = train_method + ': Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.3f}, Validation Accuracy: {:>6.3f}, Loss: {:>6.3f}'.format(epoch_i, batch_i, len(source_int_text) // reinforce_batch_size, train_acc, valid_acc, loss)
            output += ", Reward: {:>6.3f}".format(avg_reward)
            print(output)
        
    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, reinforce_save_path)
    print('Model Trained and Saved')

training...
reinforce: Epoch   0 Batch    0/2154 - Train Accuracy:  0.668, Validation Accuracy:  0.621, Loss: -1.273, Reward:  0.324
reinforce: Epoch   0 Batch    1/2154 - Train Accuracy:  0.604, Validation Accuracy:  0.621, Loss: -0.288, Reward:  0.320
reinforce: Epoch   0 Batch    2/2154 - Train Accuracy:  0.604, Validation Accuracy:  0.614, Loss: -0.260, Reward:  0.326
reinforce: Epoch   0 Batch    3/2154 - Train Accuracy:  0.603, Validation Accuracy:  0.615, Loss:  0.614, Reward:  0.308
reinforce: Epoch   0 Batch    4/2154 - Train Accuracy:  0.557, Validation Accuracy:  0.615, Loss: -1.635, Reward:  0.320
reinforce: Epoch   0 Batch    5/2154 - Train Accuracy:  0.564, Validation Accuracy:  0.614, Loss: -1.106, Reward:  0.317
reinforce: Epoch   0 Batch    6/2154 - Train Accuracy:  0.653, Validation Accuracy:  0.609, Loss: -0.441, Reward:  0.323
reinforce: Epoch   0 Batch    7/2154 - Train Accuracy:  0.650, Validation Accuracy:  0.609, Loss: -0.704, Reward:  0.319
reinforce: Epoch   0

KeyboardInterrupt: 

# Checkpoint

## Translate
This will translate `translate_sentence` from English to French.

In [23]:
checkpoint = tf.train.latest_checkpoint('checkpoints/dev')

translate_sentence = 'he saw a old yellow truck .'
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

model = RNN(1)
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, checkpoint)

    #loaded_graph = tf.Graph()
    #with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    #loader = tf.train.import_meta_graph(load_path + '.meta')
    #loader = tf.train.import_meta_graph(save_path+"/mle" + '.meta')
    
    #loader.restore(sess, load_path)

    #input_data = loaded_graph.get_tensor_by_name('input:0')
    #logits = loaded_graph.get_tensor_by_name('logits:0')
    #keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(model.inference_logits, {model.input_data: [translate_sentence], model.keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in np.argmax(translate_logits, 1)]))
print('  French Words: {}'.format([target_int_to_vocab[i] for i in np.argmax(translate_logits, 1)]))

ValueError: axis = 1 not in [-1, 1)

In [ ]:
translate_sentence = 'he saw a old yellow truck .'
translate_seq = sentence_to_seq(translate_sentence, source_vocab_to_int)

def get_samples(checkpoint, max_length, n_samples, source):
    model = RNN(n_samples)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)

        samples_out, probs_out = sess.run(
            [model.sample, model.probs],
            {model.input_data: [source]*n_samples,
            model.keep_prob: 1.})

        return samples_out, probs_out
        
checkpoint = tf.train.latest_checkpoint('checkpoints/dev')
samples_out_x, probs_out_x = get_samples(checkpoint, 1, 5, translate_seq)
print(probs_out_x)
for sample_x in samples_out_x:
    print('  French Words: {}'.format([target_int_to_vocab[i] for i in sample_x]))


In [ ]:
reinforce_epochs = 1

train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]

valid_source = helper.pad_sentence_batch(source_int_text[:batch_size])
valid_target = helper.pad_sentence_batch(target_int_text[:batch_size])

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '/mle'+'.meta')
    loader.restore(sess, load_path)

    for epoch_i in range(reinforce_epochs):

        for batch_i, (source_batch, target_batch) in enumerate(
                helper.batch_data(train_source, train_target, batch_size)):
            start_time = time.time()
            
            input_data = loaded_graph.get_tensor_by_name('input:0')
            logits = loaded_graph.get_tensor_by_name('logits:0')
            keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
            lr = loaded_graph.get_tensor_by_name('learning_rate:0')
            advantage = loaded_graph.get_tensor_by_name('advantage:0')
            targets = loaded_graph.get_tensor_by_name('targets:0')
            sequence_length = loaded_graph.get_tensor_by_name('sequence_length:0')
            
            inference_logits = loaded_graph.get_tensor_by_name('logits:0')
            cost = loaded_graph.get_tensor_by_name('optimization/cost:0')
            train_op = loaded_graph.get_operation_by_name('optimization/train_op')
            
            # temp
            #train_probs = loaded_graph.get_tensor_by_name('optimization/train_probs:0')

            
            feed_dict = {input_data: source_batch,
                         targets: target_batch,
                         lr: learning_rate,
                         sequence_length: target_batch.shape[1],
                         keep_prob: keep_probability
                        }
            
            avg_reward, loss = train_reinforce(feed_dict, advantage)

            print(loss)

            #translate_logits = sess.run(logits, {input_data: [translate_sentence], keep_prob: 1.0})[0]

## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  Additionally, the translations in this data set were made by Google translate, so the translations themselves aren't particularly good.  (We apologize to the French speakers out there!) Thankfully, for this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.